In [22]:
import re
import pymongo
import datetime
import pandas as pd
from os import path
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [14]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)
base_url = "https://beta.bls.gov/dataQuery/"
starting_url = base_url + "find?q=data+scientists&r=100&st=0"
browser.visit(starting_url)
search_html = browser.html
soup_search = bs(search_html, 'html.parser')
data_query_results = soup_search.find_all("div", class_='dq-result-item')

url_dict = {"linkID":[],"description":[],"url":[]}
data_query_df = pd.DataFrame([])

last_link_on_page = data_query_results[-1].a.get('href')

text_parent = soup_search.find_all("div", class_='dq-page-links')
pages = text_parent[0].span.text.split()[-1]

# want to put number of pages into loop boundary range(pages)
for page in range(2):
    search_html = browser.html
    soup_search = bs(search_html, 'html.parser')
    data_query_results = soup_search.find_all("div", class_='dq-result-item')
    for result in data_query_results:
        result_url = result.a.get('href')
        link_title = result.label.get_text()
        link_id = result.input.get('value')
        # print(link_title)
        # print(link_id)
        if result_url != last_link_on_page:
            url_dict["linkID"].append(link_id)
            url_dict["description"].append(link_title)
            url_dict["url"].append(result_url)
        else:
            url_dict["linkID"].append(link_id)
            url_dict["description"].append(link_title)
            url_dict["url"].append(result_url)
            next_div = soup_search.find_all("div", class_='dq-next-page')
            next_url = base_url + next_div[0].a.get('href')
            print(next_url)
    browser.visit(next_url)

url_dict_df = pd.DataFrame(url_dict)

url_dict_df

# <div id="dq-num-results-wrapper">
#            Showing Results
#            <strong>
#             1
#                             -
#                             100
#            </strong>
#            of
#            <strong>
#             5358
#            </strong>
#           </div>





====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/justin/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache
https://beta.bls.gov/dataQuery/find?st=100&r=100&q=data+scientists&more=0


,linkID,description,url
0,OEUN000000051800015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
1,OEUN000000051820015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
2,OEUN000000051700015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
3,OEUN000000051800015209803,Hourly mean wage for Data Scientists and Math...,https://beta.bls.gov/dataViewer/view/timeserie...
4,OEUN000000051800015209804,Annual mean wage for Data Scientists and Math...,https://beta.bls.gov/dataViewer/view/timeserie...
...,...,...,...
195,OEUN000000061100315209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
196,OEUN000000061130015209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
197,OEUN000000061120215209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...
198,OEUN000000061120315209801,Employment for Data Scientists and Mathematic...,https://beta.bls.gov/dataViewer/view/timeserie...


In [4]:
browser.quit()

In [15]:
text_parent = soup_search.find_all("div", class_='dq-page-links')
last_page_number = int(text_parent[0].span.text.split()[-1])
print(last_page_number)

54


In [16]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

# seriesDataTable1

for url in range(len(url_dict["url"])):
    print(url_dict["url"][url])



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/justin/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209801
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209801
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051700015209801
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209803
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209804
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209808
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051800015209813
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209803
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209804
https://beta.bls.gov/dataViewer/view/timeseries/OEUN000000051820015209808
https://beta.bls.gov/dataViewer/view/timeserie

In [43]:
# load csv
bls_state_data_df = pd.read_csv("/Users/justin/Documents/git/Project2/state_M2020_dl.csv")
# input which data points to keep 
bls_state_data_df = bls_state_data_df[["AREA_TITLE", "OCC_TITLE","OCC_CODE", "TOT_EMP", "JOBS_1000", "LOC_QUOTIENT", 'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_MEAN', 'H_PCT75', 'H_PCT90', 'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_MEAN', 'A_PCT75', 'A_PCT90']]
# generate list of states
states_list = bls_state_data_df["AREA_TITLE"].unique()
# input chosen BLS job codes
job_codes_list = ["15-1211","15-1221","15-1245","15-2031","15-2041","15-1251","15-1257","15-2098"]
# stage csv resource filenames list 
csv_list = pd.Series([object])
append_csv_list = pd.Series([object])

# create an individual csv file for each BLS job code
for job in job_codes_list:
    job_code_filter = job
    job_in_states_df = bls_state_data_df.loc[bls_state_data_df["OCC_CODE"]==job_code_filter, :]
    job_in_states_df.to_csv(f"/Users/justin/Documents/git/Project2/pandasGeneratedCSV_s/{job_code_filter}_M2020_dl.csv")
    append_csv_list = pd.Series(f"{job_code_filter}_M2020_dl.csv")
    csv_list = csv_list.append(append_csv_list, ignore_index=True)
    print(f"'{job_code_filter}_M2020_dl.csv',")

print(csv_list)

bls_state_data_df

'15-1211_M2020_dl.csv',
'15-1221_M2020_dl.csv',
'15-1245_M2020_dl.csv',
'15-2031_M2020_dl.csv',
'15-2041_M2020_dl.csv',
'15-1251_M2020_dl.csv',
'15-1257_M2020_dl.csv',
'15-2098_M2020_dl.csv',
0        <class 'object'>
1    15-1211_M2020_dl.csv
2    15-1221_M2020_dl.csv
3    15-1245_M2020_dl.csv
4    15-2031_M2020_dl.csv
5    15-2041_M2020_dl.csv
6    15-1251_M2020_dl.csv
7    15-1257_M2020_dl.csv
8    15-2098_M2020_dl.csv
dtype: object


,AREA_TITLE,OCC_TITLE,OCC_CODE,TOT_EMP,JOBS_1000,LOC_QUOTIENT,H_PCT10,H_PCT25,H_MEDIAN,H_MEAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_MEAN,A_PCT75,A_PCT90
0,Alabama,All Occupations,00-0000,"1,903,210",1000.000,1.00,8.98,11.57,17.43,22.52,27.39,41.07,"18,690","24,060","36,250","46,840","56,980","85,430"
1,Alabama,Management Occupations,11-0000,"87,110",45.772,0.80,22.95,32.37,45.73,52.90,64.58,91.89,"47,740","67,330","95,120","110,040","134,320","191,130"
2,Alabama,Chief Executives,11-1011,"1,160",0.608,0.42,23.79,47.08,77.55,84.09,#,#,"49,480","97,930","161,290","174,910",#,#
3,Alabama,General and Operations Managers,11-1021,"31,170",16.377,0.97,23.09,32.57,48.64,58.56,73.58,#,"48,030","67,740","101,170","121,800","153,050",#
4,Alabama,Legislators,11-1031,"1,150",0.604,1.64,*,*,*,*,*,*,"16,220","17,190","18,820","28,840","27,920","55,970"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36080,Virgin Islands,Crane and Tower Operators,53-7021,60,1.666,5.26,26.27,27.45,29.43,29.43,31.41,32.70,"54,630","57,100","61,220","61,220","65,340","68,020"
36081,Virgin Islands,Industrial Truck and Tractor Operators,53-7051,90,2.457,0.53,11.33,11.81,12.70,14.06,14.77,21.39,"23,570","24,560","26,410","29,240","30,720","44,480"
36082,Virgin Islands,"Laborers and Freight, Stock, and Material Move...",53-7062,600,16.748,0.83,11.22,12.01,13.96,15.01,17.49,19.94,"23,340","24,970","29,040","31,230","36,380","41,470"
36083,Virgin Islands,"Packers and Packagers, Hand",53-7064,130,3.630,0.84,11.21,11.24,11.76,12.51,12.31,15.31,"23,320","23,380","24,470","26,020","25,610","31,840"
